### 利用Dataframe整理收費名冊，並產出可列印的收費單
1. 讀入[收費資.xlsx]: 名冊、收費項目、收費期間 等內容
2. 整理成 [收費單_名冊.xlsx]：每個姓名使用1個工作表
3. 由 收費單_名冊.xlsx 產出 [收費單_列印.xlsx]: 加入表頭(該姓名對應的資料_年級、班級)，每個姓名使用1個工作表

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
with pd.ExcelFile('安親班_收費資料.xlsx') as xlsxFile:
    df_student = pd.read_excel(xlsxFile, sheet_name='學生名單', header=0)
    df_item = pd.read_excel(xlsxFile, sheet_name='收費項目', header=0) 
    df_month = pd.read_excel(xlsxFile, sheet_name='收費月份', header=0)
    df_note = pd.read_excel(xlsxFile, sheet_name='註記事項', header=0)

In [ ]:
df_student

In [ ]:
df_item

In [ ]:
df_month

In [ ]:
df_note

### Transform table

In [ ]:
df = df_month.T
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df

In [ ]:
df = pd.concat([df_item, df], axis=1)
#df.fillna(int(0), inplace=True)
df

### 收費單 名冊
#### 每位學生一個Excel工作表(所有的收費項目)

In [ ]:
name_list = df_student['姓名']
i = 0
df_note_temp = pd.DataFrame()
with pd.ExcelWriter('安親班_收費單_名冊.xlsx') as writer:
    for student in name_list:
        #將註記的內容，放在column['項目/月份'] (放在收費單的第一欄位，以對齊收費單和註記的位置)
        #收費單(資料框dataframe:df) 的第一個欄位,其名稱為'項目/月份'
        df_note_temp['項目/月份'] = df_note['註記']
        df_temp = pd.concat([df, df_note_temp], axis=0)
        #產出 收費單 excel worksheet
        df_temp.to_excel(writer, sheet_name=student, index=False)
        i += 1
df_temp

### 收費單 列印
#### 每位學生一個Excel工作表 (列印收費單)

In [ ]:
i = 0
with pd.ExcelWriter('安親班_收費單_列印.xlsx') as writer:
    for student in name_list:
        #讀入每一個學生的收費單(excel worksheet)
        df_tempt = pd.read_excel('安親班_收費單_名冊.xlsx', sheet_name=student, header=None)

        #將每一個學生的收費單轉成numpy array (context)
        content = df_temp.to_numpy() #轉成numpy array
        
        #放回一列：欄位名稱
        #因為 用 with..as..，包納read_excel後，header=None的功能 disable
        #所以轉成numpy arrany後少了原先欄位名稱(column lable)這一列
        content1 = np.insert(content, 0, [df_temp.columns], axis=0)
        
        #將每一個學生的收費表，加入表頭(學生年級、班級、姓名)
        content2 = np.insert(content1, 0, [np.NAN], axis=0) # numpy array加入第一列(表頭)
        content2[0, 0] = df_student.iloc[i]['年級'] + '年' +  df_student.iloc[i]['班級'] + '班  ' + student
        
        df2 = pd.DataFrame(content2) #轉成資料框(dataframe) (列印之學生收費單)
        
        #產出 列印收費單 excel worksheet，將 NAN 用 ''取代 
        df2.to_excel(writer, sheet_name=student, na_rep='', header=False, index=False)
        i += 1

### End of transforming

### 程式1：
* 使用pandas套件，產生收費單_名冊(Excel工作表/每位學生)

In [ ]:
import pandas as pd
import numpy as np

with pd.ExcelFile('安親班_收費資料.xlsx') as xlsxFile:
    df_student = pd.read_excel(xlsxFile, sheet_name='學生名單', header=0)
    df_item = pd.read_excel(xlsxFile, sheet_name='收費項目', header=0) 
    df_month = pd.read_excel(xlsxFile, sheet_name='收費月份', header=0)
    df_note = pd.read_excel(xlsxFile, sheet_name='註記事項', header=0)

df = df_month.T
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)

df = pd.concat([df_item, df], axis=1)
#df.fillna(int(0), inplace=True)

# 收費單 名冊
# 每位學生一個Excel工作表(所有的收費項目)
name_list = df_student['姓名']
i = 0
df_note_temp = pd.DataFrame()
with pd.ExcelWriter('安親班_收費單_名冊.xlsx') as writer:
    for student in name_list:
        #將註記的內容，放在column['項目/月份'] (放在收費單的第一欄位，以對齊收費單和註記的位置)
        #收費單(資料框dataframe:df) 的第一個欄位,其名稱為'項目/月份'
        df_note_temp['項目/月份'] = df_note['註記']
        df_temp = pd.concat([df, df_note_temp], axis=0)
        #產出 收費單 excel worksheet
        df_temp.to_excel(writer, sheet_name=student, index=False)
        i += 1

# 收費單 列印
# 每位學生一個Excel工作表 (列印收費單)
i = 0
with pd.ExcelWriter('安親班_收費單_列印.xlsx') as writer:
    for student in name_list:
        #讀入每一個學生的收費單(excel worksheet)
        df_tempt = pd.read_excel('安親班_收費單_名冊.xlsx', sheet_name=student, header=None)

        #將每一個學生的收費單轉成numpy array (context)
        content = df_temp.to_numpy() #轉成numpy array
        
        #放回一列：欄位名稱
        #因為 用 with..as..，包納read_excel後，header=None的功能 disable
        #所以轉成numpy arrany後少了原先欄位名稱(column lable)這一列
        content1 = np.insert(content, 0, [df_temp.columns], axis=0)
        
        #將每一個學生的收費表，加入表頭(學生年級、班級、姓名)
        content2 = np.insert(content1, 0, [np.NAN], axis=0) # numpy array加入第一列(表頭)
        content2[0, 0] = df_student.iloc[i]['年級'] + '年' +  df_student.iloc[i]['班級'] + '班  ' + student
        
        df2 = pd.DataFrame(content2) #轉成資料框(dataframe) (列印之學生收費單)
        
        #產出 列印收費單 excel worksheet，將 NAN 用 ''取代 
        df2.to_excel(writer, sheet_name=student, na_rep='', header=False, index=False)
        i += 1

### 程式2：
* 使用openpyxl套件，產生收費單

In [ ]:
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side

with pd.ExcelFile('安親班_收費資料.xlsx') as xlsxFile:
    df_student = pd.read_excel(xlsxFile, sheet_name='學生名單', header=0)
    df_month = pd.read_excel(xlsxFile, sheet_name='收費月份', header=0)
 
# 定義表頭的樣式
style_head = {
    "border": Border(left=Side(style='medium', color='FF000000'), 
                     right=Side(style='medium', color='FF000000'),
                     top=Side(style='medium', color='FF000000'), 
                     bottom=Side(style='medium', color='FF000000')),
    "fill": PatternFill("solid", fgColor="9AFF9A"),
    "font": Font(color="000000", bold=True, name="標楷體", size=14),
    "alignment": Alignment(horizontal="center", vertical="center")
}

# 定義表內容樣式
style_content = {
    "border": Border(left=Side(style='thin', color='FF000000'), 
                     right=Side(style='thin', color='FF000000'),
                     top=Side(style='thin', color='FF000000'), 
                     bottom=Side(style='thin', color='FF000000')),
    "alignment": Alignment(horizontal='left', vertical='center'),
    "font": Font(name="標楷體")}

name_list = df_student['姓名']
with pd.ExcelFile('安親班_收費單_列印.xlsx') as xlsxFile:
    wb = load_workbook(xlsxFile)
    for student in name_list:
        #讀入每一個學生的收費單(excel worksheet)       
        ws = wb[student]
        
        cols = []
        for col in ws.iter_cols():
            cols.append(col)
        rows = []
        for row in ws.iter_rows():
            rows.append(row)
        
        # merge cells according to the table of each sheet
        # 年級, 班級, 姓名
        ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=len(cols))       
        #ws['A1'].border = style_head['border']
        ws['A1'].fill = style_head['fill']
        ws['A1'].font = style_head['font']
        ws['A1'].alignment = style_head['alignment']    
        ws.row_dimensions[1].height = 30
        
        row, col = df_month.shape
        
        # 畫表格
        for j in range(2, len(rows)-row+2):
            for k in range(len(cols)):
                ws.cell(row=j, column=k+1).border = style_head['border']
                ws.cell(row=j, column=k+1).font = style_content['font']
                ws.cell(row=j, column=k+1).alignment = style_content['alignment']
            ws.column_dimensions['A'].width = 31
            ws.row_dimensions[j].height = 20
            
        # 註記事項               
        for i in range(row-1):
            ws.merge_cells(start_row=len(rows)-i, start_column=1, end_row=len(rows)-i, end_column=len(cols))
            #ws.cell(row=len(rows)-i, column=1).border = style_content['border']
            ws.cell(row=len(rows)-i, column=1).font = style_content['font']
            ws.cell(row=len(rows)-i, column=1).alignment = style_content['alignment']
            ws.row_dimensions[len(rows)-i].height = 15
        
wb.save('收費單.xlsx')    

####  程式2-1
使用openpyxl套件，產生收費單(合併工作表)

In [45]:
###
# 程式 2
##        
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side

with pd.ExcelFile('安親班_收費資料.xlsx') as xlsxFile:
    df_student = pd.read_excel(xlsxFile, sheet_name='學生名單', header=0)
    df_month = pd.read_excel(xlsxFile, sheet_name='收費月份', header=0)
 
# 定義表頭的樣式
style_head = {
    "border": Border(left=Side(style='medium', color='FF000000'), 
                     right=Side(style='medium', color='FF000000'),
                     top=Side(style='medium', color='FF000000'), 
                     bottom=Side(style='medium', color='FF000000')),
    "fill": PatternFill("solid", fgColor="9AFF9A"),
    "font": Font(color="000000", bold=True, name="標楷體", size=14),
    "alignment": Alignment(horizontal="center", vertical="center")
}

# 定義表內容樣式
style_content = {
    "border": Border(left=Side(style='thin', color='FF000000'), 
                     right=Side(style='thin', color='FF000000'),
                     top=Side(style='thin', color='FF000000'), 
                     bottom=Side(style='thin', color='FF000000')),
    "alignment": Alignment(horizontal='left', vertical='center'),
    "font": Font(name="標楷體")}

name_list = df_student['姓名']   
##
# merge worksheet
#
   
wb_source = load_workbook('收費單.xlsx')   
length = len(name_list)
noMerge = 4 # number of worksheets to be merged

# 根據 noMerge 值，分群(noMerge+1個新的工作表)
base = 0
for i in range(length//noMerge):
    base = i * noMerge # base index of worksheet
    rowIdx = 0 # inital row index of the new worksheet 
    # 合併 noMerge 個工作表
    for idx in range(noMerge):
        sheetName = name_list[base+idx]
        ws = wb_source[sheetName]
        if idx == 0:
            sheetName_new = sheetName + '_merged'
            wb_source.create_sheet(sheetName_new)
            ws_target = wb_source[sheetName_new]

        # copy each row in worksheet to the target merged workshee     
        for index_r, row in enumerate(ws.rows, start=1):
            #print(row, index_r)
            for index_c, col in enumerate(row, start=1):
                #print(col, index_c)
                x1 = col.value
                ws_target.cell(row=rowIdx + index_r, column=index_c).value = x1
        
        # update row index
        rowIdx = rowIdx + index_r+1
    print(i)

base = (i+1)*noMerge # base index of worksheet
rowIdx = 0 # row index of worksheet
print(base, rowIdx)
# 根據 noMerge 值，分群(noMerge+1個新的工作表)最後1個工作表
for idx in range(length%noMerge):
    sheetName = name_list[base+idx]
    ws = wb_source[sheetName]
    if idx == 0:
        sheetName_new = sheetName + '_merged'
        wb_source.create_sheet(sheetName_new)
        ws_target = wb_source[sheetName_new]

    # copy each row in worksheet to the target merged workshee     
    for index_r, row in enumerate(ws.rows, start=1):
        #print(row, index_r)
        for index_c, col in enumerate(row, start=1):
            #print(col, index_c)
            x1 = col.value
            ws_target.cell(row=rowIdx + index_r, column=index_c).value = x1
        
    # update row index
    rowIdx = rowIdx + index_r+1

# finally
wb_source.save('收費單_合併.xlsx')

0
1
2
12 0


### 刪除不必要的檔案

In [ ]:
import os

file_1 = r'安親班_收費單_名冊.xlsx'
file_2 = r'安親班_收費單_列印.xlsx'

try:
    os.remove(file_1)
    os.remove(file_2)
except OSError as e:
    print(e)
else:
    print("Files are deleted successfully")

#### 程式3：
讀取安親班_收費資料，使用pandas、openpyxl、win32com.client套件，產生收費單(名冊:Excel工作表/每位學生)、收費單(列印:PDF 1個學生/頁)

In [ ]:
###
# 程式 1
##
import pandas as pd
import numpy as np

with pd.ExcelFile('安親班_收費資料.xlsx') as xlsxFile:
    df_student = pd.read_excel(xlsxFile, sheet_name='學生名單', header=0)
    df_item = pd.read_excel(xlsxFile, sheet_name='收費項目', header=0) 
    df_month = pd.read_excel(xlsxFile, sheet_name='收費月份', header=0)
    df_note = pd.read_excel(xlsxFile, sheet_name='註記事項', header=0)

df = df_month.T
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)

df = pd.concat([df_item, df], axis=1)
#df.fillna(int(0), inplace=True)

# 收費單 名冊
# 每位學生一個Excel工作表(所有的收費項目)
name_list = df_student['姓名']
i = 0
df_note_temp = pd.DataFrame()
with pd.ExcelWriter('安親班_收費單_名冊.xlsx') as writer:
    for student in name_list:
        #將註記的內容，放在column['項目/月份'] (放在收費單的第一欄位，以對齊收費單和註記的位置)
        #收費單(資料框dataframe:df) 的第一個欄位,其名稱為'項目/月份'
        df_note_temp['項目/月份'] = df_note['註記']
        df_temp = pd.concat([df, df_note_temp], axis=0)
        #產出 收費單 excel worksheet
        df_temp.to_excel(writer, sheet_name=student, index=False)
        i += 1

# 收費單 列印
# 每位學生一個Excel工作表 (列印收費單)
i = 0
with pd.ExcelWriter('安親班_收費單_列印.xlsx') as writer:
    for student in name_list:
        #讀入每一個學生的收費單(excel worksheet)
        df_tempt = pd.read_excel('安親班_收費單_名冊.xlsx', sheet_name=student, header=None)

        #將每一個學生的收費單轉成numpy array (context)
        content = df_temp.to_numpy() #轉成numpy array
        
        #放回一列：欄位名稱
        #因為 用 with..as..，包納read_excel後，header=None的功能 disable
        #所以轉成numpy arrany後少了原先欄位名稱(column lable)這一列
        content1 = np.insert(content, 0, [df_temp.columns], axis=0)
        
        #將每一個學生的收費表，加入表頭(學生年級、班級、姓名)
        content2 = np.insert(content1, 0, [np.NAN], axis=0) # numpy array加入第一列(表頭)
        content2[0, 0] = df_student.iloc[i]['年級'] + '年' +  df_student.iloc[i]['班級'] + '班  ' + student
        
        df2 = pd.DataFrame(content2) #轉成資料框(dataframe) (列印之學生收費單)
        
        #產出 列印收費單 excel worksheet，將 NAN 用 ''取代 
        df2.to_excel(writer, sheet_name=student, na_rep='', header=False, index=False)
        i += 1

###
# 程式 2
##        
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side

with pd.ExcelFile('安親班_收費資料.xlsx') as xlsxFile:
    df_student = pd.read_excel(xlsxFile, sheet_name='學生名單', header=0)
    df_month = pd.read_excel(xlsxFile, sheet_name='收費月份', header=0)
 
# 定義表頭的樣式
style_head = {
    "border": Border(left=Side(style='medium', color='FF000000'), 
                     right=Side(style='medium', color='FF000000'),
                     top=Side(style='medium', color='FF000000'), 
                     bottom=Side(style='medium', color='FF000000')),
    "fill": PatternFill("solid", fgColor="9AFF9A"),
    "font": Font(color="000000", bold=True, name="標楷體", size=14),
    "alignment": Alignment(horizontal="center", vertical="center")
}

# 定義表內容樣式
style_content = {
    "border": Border(left=Side(style='thin', color='FF000000'), 
                     right=Side(style='thin', color='FF000000'),
                     top=Side(style='thin', color='FF000000'), 
                     bottom=Side(style='thin', color='FF000000')),
    "alignment": Alignment(horizontal='left', vertical='center'),
    "font": Font(name="標楷體")}

name_list = df_student['姓名']
with pd.ExcelFile('安親班_收費單_列印.xlsx') as xlsxFile:
    wb = load_workbook(xlsxFile)
    for student in name_list:
        #讀入每一個學生的收費單(excel worksheet)       
        ws = wb[student]
        
        cols = []
        for col in ws.iter_cols():
            cols.append(col)
        rows = []
        for row in ws.iter_rows():
            rows.append(row)
        
        # merge cells according to the table of each sheet
        # 年級, 班級, 姓名
        ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=len(cols))       
        #ws['A1'].border = style_head['border']
        ws['A1'].fill = style_head['fill']
        ws['A1'].font = style_head['font']
        ws['A1'].alignment = style_head['alignment']    
        ws.row_dimensions[1].height = 30
        
        row, col = df_month.shape
        
        # 畫表格
        for j in range(2, len(rows)-row+2):
            for k in range(len(cols)):
                ws.cell(row=j, column=k+1).border = style_head['border']
                ws.cell(row=j, column=k+1).font = style_content['font']
                ws.cell(row=j, column=k+1).alignment = style_content['alignment']
            ws.column_dimensions['A'].width = 31
            ws.row_dimensions[j].height = 20
            
        # 註記事項               
        for i in range(row-1):
            ws.merge_cells(start_row=len(rows)-i, start_column=1, end_row=len(rows)-i, end_column=len(cols))
            #ws.cell(row=len(rows)-i, column=1).border = style_content['border']
            ws.cell(row=len(rows)-i, column=1).font = style_content['font']
            ws.cell(row=len(rows)-i, column=1).alignment = style_content['alignment']
            ws.row_dimensions[len(rows)-i].height = 15
        
wb.save('收費單.xlsx') 
wb.close()

###
# 刪除不必要的檔案
##

import os

file_1 = r'安親班_收費單_名冊.xlsx'
file_2 = r'安親班_收費單_列印.xlsx'

try:
    os.remove(file_1)
    os.remove(file_2)
except OSError as e:
    print(e)
else:
    print("Files are deleted successfully")

###
# convet to pdf format
##

import win32com.client

o = win32com.client.Dispatch("Excel.Application")
o.Visible = True #False
wb_p_path = 'c:\\pywhyang\\student_care\\收費單.xlsx'
wb_p = o.Workbooks.Open(wb_p_path)

ws_index_list = name_list #[1,2,3,4,5,6] #say you want to print these sheets
path_to_pdf =  'c:\\pywhyang\\student_care\\收費單.pdf' 
#wb_p.WorkSheets(ws_index_list).Select()
wb_p.Sheets(ws_index_list).Select()

wb_p.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)
wb_p.Close(True)

#### 程式4：(simplifying)
讀取安親班_收費資料，使用pandas、openpyxl、win32com.client套件，產生收費單(名冊:Excel工作表/每位學生)、收費單(列印:PDF 幾個學生/頁)

In [ ]:
###
# 程式 1
##
import pandas as pd
import numpy as np

with pd.ExcelFile('安親班_收費資料.xlsx') as xlsxFile:
    df_student = pd.read_excel(xlsxFile, sheet_name='學生名單', header=0)
    df_item = pd.read_excel(xlsxFile, sheet_name='收費項目', header=0) 
    df_month = pd.read_excel(xlsxFile, sheet_name='收費月份', header=0)
    df_note = pd.read_excel(xlsxFile, sheet_name='註記事項', header=0)

df = df_month.T
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)

df = pd.concat([df_item, df], axis=1)
#df.fillna(int(0), inplace=True)

# 收費單 名冊
# 每位學生一個Excel工作表(所有的收費項目)
name_list = df_student['姓名']
i = 0
df_note_temp = pd.DataFrame()
with pd.ExcelWriter('安親班_收費單_名冊.xlsx') as writer:
    for student in name_list:
        #將註記的內容，放在column['項目/月份'] (放在收費單的第一欄位，以對齊收費單和註記的位置)
        #收費單(資料框dataframe:df) 的第一個欄位,其名稱為'項目/月份'
        df_note_temp['項目/月份'] = df_note['註記']
        df_temp = pd.concat([df, df_note_temp], axis=0)
        #產出 收費單 excel worksheet
        df_temp.to_excel(writer, sheet_name=student, index=False)
        i += 1

# 收費單 列印
# 每位學生一個Excel工作表 (列印收費單)
i = 0
with pd.ExcelWriter('安親班_收費單_列印.xlsx') as writer:
    for student in name_list:
        #讀入每一個學生的收費單(excel worksheet)
        df_tempt = pd.read_excel('安親班_收費單_名冊.xlsx', sheet_name=student, header=None)

        #將每一個學生的收費單轉成numpy array (context)
        content = df_temp.to_numpy() #轉成numpy array
        
        #放回一列：欄位名稱
        #因為 用 with..as..，包納read_excel後，header=None的功能 disable
        #所以轉成numpy arrany後少了原先欄位名稱(column lable)這一列
        content1 = np.insert(content, 0, [df_temp.columns], axis=0)
        
        #將每一個學生的收費表，加入表頭(學生年級、班級、姓名)
        content2 = np.insert(content1, 0, [np.NAN], axis=0) # numpy array加入第一列(表頭)
        content2[0, 0] = df_student.iloc[i]['年級'] + '年' +  df_student.iloc[i]['班級'] + '班  ' + student
        
        df2 = pd.DataFrame(content2) #轉成資料框(dataframe) (列印之學生收費單)
        
        #產出 列印收費單 excel worksheet，將 NAN 用 ''取代 
        df2.to_excel(writer, sheet_name=student, na_rep='', header=False, index=False)
        i += 1

###
# 程式 2
##        
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side

with pd.ExcelFile('安親班_收費資料.xlsx') as xlsxFile:
    df_student = pd.read_excel(xlsxFile, sheet_name='學生名單', header=0)
    df_month = pd.read_excel(xlsxFile, sheet_name='收費月份', header=0)
 
# 定義表頭的樣式
style_head = {
    "border": Border(left=Side(style='medium', color='FF000000'), 
                     right=Side(style='medium', color='FF000000'),
                     top=Side(style='medium', color='FF000000'), 
                     bottom=Side(style='medium', color='FF000000')),
    "fill": PatternFill("solid", fgColor="9AFF9A"),
    "font": Font(color="000000", bold=True, name="標楷體", size=14),
    "alignment": Alignment(horizontal="center", vertical="center")
}

# 定義表內容樣式
style_content = {
    "border": Border(left=Side(style='thin', color='FF000000'), 
                     right=Side(style='thin', color='FF000000'),
                     top=Side(style='thin', color='FF000000'), 
                     bottom=Side(style='thin', color='FF000000')),
    "alignment": Alignment(horizontal='left', vertical='center'),
    "font": Font(name="標楷體")}

name_list = df_student['姓名']
with pd.ExcelFile('安親班_收費單_列印.xlsx') as xlsxFile:
    wb = load_workbook(xlsxFile)
    for student in name_list:
        #讀入每一個學生的收費單(excel worksheet)       
        ws = wb[student]
        
        cols = []
        for col in ws.iter_cols():
            cols.append(col)
        rows = []
        for row in ws.iter_rows():
            rows.append(row)
        
        # merge cells according to the table of each sheet
        # 年級, 班級, 姓名
        ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=len(cols))       
        #ws['A1'].border = style_head['border']
        ws['A1'].fill = style_head['fill']
        ws['A1'].font = style_head['font']
        ws['A1'].alignment = style_head['alignment']    
        ws.row_dimensions[1].height = 30
        
        row, col = df_month.shape
        
        # 畫表格
        for j in range(2, len(rows)-row+2):
            for k in range(len(cols)):
                ws.cell(row=j, column=k+1).border = style_head['border']
                ws.cell(row=j, column=k+1).font = style_content['font']
                ws.cell(row=j, column=k+1).alignment = style_content['alignment']
            ws.column_dimensions['A'].width = 31
            ws.row_dimensions[j].height = 20
            
        # 註記事項               
        for i in range(row-1):
            ws.merge_cells(start_row=len(rows)-i, start_column=1, end_row=len(rows)-i, end_column=len(cols))
            #ws.cell(row=len(rows)-i, column=1).border = style_content['border']
            ws.cell(row=len(rows)-i, column=1).font = style_content['font']
            ws.cell(row=len(rows)-i, column=1).alignment = style_content['alignment']
            ws.row_dimensions[len(rows)-i].height = 15
        
    wb.save('收費單.xlsx') 
    wb.close()     
        
###
# 刪除不必要的檔案
##

import os

file_1 = r'安親班_收費單_名冊.xlsx'
file_2 = r'安親班_收費單_列印.xlsx'

try:
    os.remove(file_1)
    os.remove(file_2)
except OSError as e:
    print(e)
else:
    print("Files are deleted successfully")


###
# convet to pdf format
##

import win32com.client

o = win32com.client.Dispatch("Excel.Application")
o.Visible = True #False
wb_p_path = 'c:\\pywhyang\\student_care\\收費單.xlsx'
wb_p = o.Workbooks.Open(wb_p_path)

ws_index_list = name_list #[1,2,3,4,5,6] #say you want to print these sheets
path_to_pdf =  'c:\\pywhyang\\student_care\\收費單.pdf' 
#wb_p.WorkSheets(ws_index_list).Select()
wb_p.Sheets(ws_index_list).Select()

wb_p.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)
wb_p.Close(True)

###
# merge three pages in pdf 
##
#import win32com.client as win32

length = len(name_list)
for i in range(length//3):
    j = i * 3
    print(name_list[j])
    print(name_list[j+1])
    print(name_list[j+2])
    print('-----', i)
    
print('*****-----')
k = len(name_list) - ((i+1) * 3)
print(i, k)
n=(i+1)*3
for m in range(k):
    print(name_list[n+m])
print('#######')

from openpyxl import load_workbook
import os
from openpyxl import Workbook

def read_ws(wkbk, name, rows):
    ws = wkbk[name]
    #rows = []
    for row in ws.iter_rows():
        rows.append(row)
    return rows

with pd.ExcelFile('c:\\pywhyang\\student_care\\收費單.xlsx') as xlsxFile:
    # construct the workbook in which every 3 students' sheets are combined as as one worksheet  
    wb_3pages = Workbook()
    ws1 = wb.create_sheet('銀行', index=0) 創造合併之後的sheet表的名字
    
    wb_3pages = load_workbook(xlsxFile)
    length = len(name_list)    
    for i in range(length//3):
        j = i * 3
        print(name_list[j])
        title = str(name_list[j]+name_list[j+1]+name_list[j+2])
        ws_3pages_sheet = wb_3pages.create_sheet(title)
        ws_3pages_sheet = read_ws(wkbk=wb_3pages, name=name_list[j], rows=ws_3pages_sheet)
        print(name_list[j+1])
        ws_3pages_sheet = read_ws(wkbk=wb_3pages, name=name_list[j+1], rows=ws_3pages_sheet)
        print(name_list[j+2])
        ws_3pages_sheet = read_ws(wkbk=wb_3pages, name=name_list[j+2], rows=ws_3pages_sheet)
        print('-----', i)
        wb_3pages[title]=ws_3pages_sheet
    
    print('*****-----')
    k = len(name_list) - ((i+1) * 3)
    print(i, k)
    n=(i+1)*3
    for m in range(k):
        print(name_list[n+m])
    print('#######')
    wb_3pages.save('收費單_3pages.xlsx')
   

In [ ]:
name1 = 'A'
name2 = 'B'
name3 = 'C'
names = list([name1, name2, name3])
print(names)
names[0]

In [ ]:
wb.close()

#### combine all worksheets into one worksheet

In [ ]:
import pandas as pd

df_student = pd.read_excel('c:\\pywhyang\\student_care\\安親班_收費資料.xlsx', sheet_name='學生名單', header=0)
name_list = df_student['姓名']

df_merge = pd.DataFrame()
for name in name_list:
    df_payment = pd.read_excel('c:\\pywhyang\\student_care\\收費單.xlsx', sheet_name=name, header=None)   
    df_merge = pd.concat([df_merge, df_payment], axis=0)
    
#df_merge.drop(df_merge.index[0], inplace=True)
df_merge.to_excel('c:\\pywhyang\\student_care\\收費單_pages.xlsx', sheet_name='whole', index=False)
df_merge.head(20)

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl import Workbook
  
def read_ws(wb, name, targetsheet):
    sheet = wb[name]   
    for index, row in enumerate(sheet.rows):
        for i_idx in index:
            idx += i_idx 
            print(idx)
            targetsheet.insert_rows(idx)
            j_idx = 0
            for item in row:
                print(item.value)
                j_idx += 1
                #content.append(item.value)
                targetsheet.cell(row=idx, column=j_idx).value = item.value
    return targetsheet

df_student = pd.read_excel('c:\\pywhyang\\student_care\\安親班_收費資料.xlsx', sheet_name='學生名單', header=0)
name_list = df_student['姓名']

with pd.ExcelFile('c:\\pywhyang\\student_care\\收費單.xlsx') as xlsxFile:
    # construct the workbook in which every 3 students' sheets are combined as as one worksheet  
    wb_3pages = Workbook()
    ws_3pages = wb_3pages.create_sheet('收費單', index=0)
    
    # go through every one worksheet in terms of each student
    wb = load_workbook(xlsxFile)
    
    global idx
    idx = 1
    for name in name_list:
        ws_3pages = read_ws(wb, name, ws_3pages)
    
    wb_3pages.save('c:\\pywhyang\\student_care\\收費單_pages.xlsx') 

### example

#### merge worksheets (with same structure) via openpyxl package

In [ ]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
# time 2018-11-23
# Author fcj
# message 處理資料

from openpyxl import load_workbook
import os
from openpyxl import Workbook
from openpyxl.drawing.image import Image  # 注意當你想往excel寫入圖片時，可使用這個方法。


def del_excel():
    file_list = os.walk('D:/集合/銀行/')  # 獲取這個資料夾下所有的excel文件。

    for file in file_list:
    
        path_list = file[2]
        wb = Workbook()
        ws1 = wb.create_sheet('銀行', index=0) 創造合併之後的sheet表的名字
        title_list = ['名字','個人照片', '二維碼']  # sheet表的表頭建立
        ws1.append(title_list)
        
        for ph in path_list:  #  迭代遍歷所有excel 文件
            print('ph %s' % ph)
            ph = ph.replace('~$', '')
            if str(ph).endswith('sx'):
                res = find_excel(path='D:/集合/銀行/' + ph)   # 構建單個文件路徑
                ws1.append(res)
                wb.save('D:/FX.xlsx')


def find_excel(path):  #  獲取excel文件內容的方法
    wb = load_workbook(path)
    sheet = wb.active
    content = []
    
    for index, row in enumerate(sheet.rows):
        if index > 0:  # 因為 index = 0時，獲取到的為表格的值。
            # print(row)
            for item in row:   #  這個迭代無特殊情況 可簡化一下
                # print(item.value)
                content.append(item.value)    #  注意，獲取到的是一行的值

    return content


if __name__ == '__main__':
    del_excel()

### read worksheet then save as pdf

In [ ]:
import win32com.client

o = win32com.client.Dispatch("Excel.Application")
o.Visible = False
wb_path = r'C:\\pywhyang\\student_care\\收費單.xlsx'
wb_p = o.Workbooks.Open(wb_path)

ws_index_list = [1,2,3,4,5,6] #name_list 

path_to_pdf =  'c:\\pywhyang\\student_care\\收費單.pdf'
wb_p.Sheets(ws_index_list).Select()
#wb_p.WorkSheets(ws_index_list).Select()
wb_p.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)
wb_p.Close(True)

In [ ]:
import win32com.client
import os
import traceback
import sys
import easygui

def gui():
    msg = ""
    title = "ETP"
    fieldNames = ["Path", "Worksheet(s)", "Print Area(s)", "Paper Size(s)"]
    fieldValues = [".txt or .xlsx", "1,3-5,6,7,11-9", "A1:G57,B4:Z57,A2:G56,A2:G56,A5:J72", "1,2,1,1,2"]
    fieldValues = easygui.multenterbox(msg, title, fieldNames, fieldValues)
    return fieldValues

def get_pathlist(path):
    filetype = os.path.splitext(path)[1]
    
    if filetype == ".txt":
        athlist = [file for file in open(path).readlines]
    elif filetype == ".xlsx" or filetype == ".xls":
        pathlist = path
    else:
        #error
        pass
    return pathlist

def get_wslist(worksheets):
    try:
        spl = worksheets.split(',')
    except:
        raise()
    
    p1 = re.compile([0-9]+)
    p2 = re.compile([0-9]+[-][0-9]+)

    ws_list = []

    for ws in spl:
        if "-" not in ws:
            if p1.match(ws) != None:
                ws_list.append(ws)
        elif "-" in ws:
            if p2.match(ws) != None:
                f = ws.split('-')[0]
                l = ws.split('-')[:-1]
                if f < l:
                    for n in range(f,l,1):
                        ws_list.append(n)
                elif f > l:
                    for n in range(l,f,-1):
                        ws_list.append(n)
                elif f == l:
                    ws_list.append(ws)
                else:
                    raise()
        else:
            raise()

def get_palist(print_areas):
    try:
        spl = print_areas.split(',')
    except:
        raise()

    p = re.compile(r'[A-z]+[0-9]+[:][A-z]+[0-9]+', re.IGNORECASE)

    for pa in spl:
        if p.match(pa) == None:
            raise()

    return spl

def get_pslist(page_sizes):
    try:
        spl = page_sizes.split(',')
    except:
        raise()

    for ps in spl:
        if ps != "1" and ps != "2":
            raise()

    return spl

def open_wb(wb_path):
    o = win32com.client.Dispatch("Excel.Application")
    o.Visible = False
    wb = o.Workbooks.Open(wb_path)
    return wb

def print_setup(ws, print_area):
    ws.PageSetup.Zoom = False
    ws.PageSetup.FitToPagesTall = 1
    ws.PageSetup.FitToPagesWide = 1
    ws.PageSetup.PrintArea = print_area

    #ws.PageSetup.XlPaper11x17
    #ws.PageSetup.XlPaperLetter

    #ws.PageSetup.XlLandscape
    #ws.PageSetup.XlPortrait

# def print_single_ws(wb, wb_path, ws_index, print_area = None, pdf_path = None):
    # if pdf_path is None:
        # pdf_path = os.path.dirname(wb_path) + "\\" + os.path.splitext(os.path.basename(wb_path))[0]

    # if print_area is not None:
        # print_setup(ws, print_area)

    # ws = wb.Worksheets[ws_index]
    # ws.ExportAsFixedFormat(0, pdf_path)

def print_ws(wb, wb_path, ws_index_list, print_area_list, pdf_path = None):
    if pdf_path is None:
        pdf_path = os.path.dirname(wb_path) + "\\" + os.path.splitext(os.path.basename(wb_path))[0]

    if print_area is not None:
        for index in ws_index_list:
            ws = wb.Worksheets[index-1]
            print_setup(ws, print_area, page_size)

    wb.WorkSheets(ws_index_list).Select()
    wb.ActiveSheet.ExportAsFixedFormat(0, pdf_path)
    wb.WorkSheets(1).Select()

def close(wb):
    wb.Close(True)

def manager():
    input_values = gui()
    path = input_values[0]
    worksheets = input_values[1]
    print_areas = input_values[2]
    page_sizes = input_values[3]

    pathlist = get_pathlist(path)
    ws_list = get_wslist(worksheets)
    pa_list = get_palist(print_areas)
    ps_list = get_pslist(page_sizes)

    for path in pathlist:
        wb = open_wb(path)
        print_ws(wb, path, ws_list, pa_list, ps_list)

if __name__ == "__main__":
    manager()